In [1]:
%matplotlib inline

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from cv2 import VideoWriter, VideoWriter_fourcc
import pandas as pd
import torch
from scipy import signal, ndimage, spatial
from scipy.signal import correlate
from scipy.interpolate import CubicSpline
from scipy.ndimage import gaussian_filter
import math 

from scipy.interpolate import interp1d
import scipy.stats as stats
from scipy.stats import spearmanr

## Read the SLP Scores 

In [3]:
path = r'/home/deniz/Code/ClinicalScoreProject/NeuroFace_Dataset/ALS/SLP_Assessment_ALS.csv'

df = pd.read_csv(path)

df = df.drop('Unnamed: 8', axis=1)

print(len(df))
print(df.columns)
df.head()

76
Index(['File Name', 'Subject ID', 'Symmetry (SLP1)', 'ROM (SLP1)',
       'Speed (SLP1)', 'Variability (SLP1)', 'Fatigue (SLP1)', 'Tot (SLP1)',
       'Symmetry (SLP2)', 'ROM (SLP2)', 'Speed (SLP2)', 'Variability (SLP2)',
       'Fatigue (SLP2)', 'Tot (SLP2)'],
      dtype='object')


,File Name,Subject ID,Symmetry (SLP1),ROM (SLP1),Speed (SLP1),Variability (SLP1),Fatigue (SLP1),Tot (SLP1),Symmetry (SLP2),ROM (SLP2),Speed (SLP2),Variability (SLP2),Fatigue (SLP2),Tot (SLP2)
0,A002_02_BBP_NORMAL_color.avi,A002,1,1,1,1,1,5,2,1,1,1,1,6
1,A002_02_DDK_PATAKA_color.avi,A002,1,1,1,1,1,5,2,2,1,2,2,9
2,A002_02_DDK_PA_color.avi,A002,1,1,1,1,1,5,2,2,1,2,2,9
3,A002_02_NSM_BLOW_color.avi,A002,2,1,1,2,1,7,1,2,2,2,1,8
4,A002_02_NSM_KISS_color.avi,A002,2,1,1,1,4,9,1,1,1,1,3,7


In [4]:
df.groupby(['Subject ID','File Name'])
#.plot(kind='scatter',x='Tot (SLP1)',y='Tot (SLP2)')

In [7]:
path2 = r'/home/deniz/Code/ClinicalScoreProject/NeuroFace_Dataset/ALS/VID_DATASET_Clinical information_ALS.csv'

df2 = pd.read_csv(path2)

print(len(df2))
print(df2.columns)
df2.head()

10
Index(['SubjectID', 'AgeSession', 'Gender', 'DisDurDiag', 'AgeDiag',
       'DisDurON', 'AgeOnset', 'SiteOnset', 'ALSFRSTot', 'ALSFRSBulb',
       'Intell (%)', 'SpeakRate (words/min)'],
      dtype='object')


,SubjectID,AgeSession,Gender,DisDurDiag,AgeDiag,DisDurON,AgeOnset,SiteOnset,ALSFRSTot,ALSFRSBulb,Intell (%),SpeakRate (words/min)
0,A002,62,F,19,60,31,59,Lower limbs,31,12,100,151.72
1,A006,64,F,20,62,109,55,Upper/Lower limbs,26,12,96.36,122.68
2,A008,55,M,15,54,28,53,Upper limbs,39,11,91.82,143.17
3,A009,45,F,29,42,41,41,Lower limbs,26,7,28.18,63.28
4,A010,68,M,10,67,26,66,Upper limbs,40,12,87.27,88.61
